<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/RSI_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime
import yfinance as yf

pd.options.display.float_format = '{:.2f}'.format

close = 'Close'
# close = 'Adj Close'

etf_kospi = ['^KS11']
etf_euro = ['^STOXX50E']

etf_order = etf_kospi

df_etf = yf.download(etf_order)[close]
if isinstance(df_etf, pd.Series):
  df_etf = df_etf.to_frame()

display(df_etf)

[*********************100%%**********************]  1 of 1 completed


,Close
Date,
1996-12-11,704.68
1996-12-12,689.38
1996-12-13,689.07
1996-12-16,673.92
1996-12-17,663.35
...,...
2023-09-27,2465.07
2023-10-04,2405.69
2023-10-05,2403.60


In [ ]:
date_index = df_etf.index

def calcRSI(df, period):
    # 가격 차이 계산
    price_diff = df.diff(1)

    # 상승일과 하락일 계산
    U = np.where(price_diff >= 0, price_diff, 0)
    D = np.where(price_diff < 0, -price_diff, 0)

    # EMA(지수 가중 이동 평균) 계산
    # AU = pd.DataFrame(U, index=date_index).ewm(span=period, adjust=False).mean()
    # AD = pd.DataFrame(D, index=date_index).ewm(span=period, adjust=False).mean()
    AU = pd.DataFrame(U, index=date_index).rolling(window=period).mean()
    AD = pd.DataFrame(D, index=date_index).rolling(window=period).mean()

    # RSI 계산
    RS = AU / AD
    RSI = 100 - (100 / (1 + RS))

    return RSI

# RSI 계산
df_rsi = calcRSI(df_etf, 14).dropna()
df_rsi.columns = [close]

df_etf_rsi = pd.merge(df_etf, df_rsi, on='Date', how='right')
df_etf_rsi.columns = [close, 'RSI']

# 결과 출력
display(df_etf_rsi)

,Close,RSI
Date,,
1997-01-06,626.87,25.97
1997-01-07,611.05,23.66
1997-01-08,621.41,30.33
1997-01-09,639.87,37.12
1997-01-10,648.69,43.17
...,...,...
2023-09-27,2465.07,32.67
2023-10-04,2405.69,26.24
2023-10-05,2403.60,23.73


In [ ]:
df_etf_rsi_filtered = df_etf_rsi[::21]

display(df_etf_rsi_filtered.head(30))

,Close,RSI
Date,,
1997-01-06,626.87,25.97
1997-02-04,695.95,48.58
1997-03-06,678.11,27.74
1997-04-04,687.42,60.91
1997-05-07,689.10,40.11
1997-06-09,775.01,74.65
1997-07-08,767.74,47.19
1997-08-07,745.35,48.51
1997-09-08,697.98,28.87


In [ ]:
money = 10_000_000

df_shares = pd.DataFrame(index=df_etf_rsi_filtered.index, columns=['shares'])
df_money = pd.DataFrame(index=df_etf_rsi_filtered.index, columns=['money'])
prev_index_num = 0

for index, row in df_etf_rsi_filtered.iterrows():
  date_time = index.strftime('%Y-%m-%d')
  rsi = row['RSI']
  price_close = row[close]

  prev_index_num = df_etf_rsi_filtered.index.to_list().index(index) - 1

  if rsi <= 30:
    shares = 0

    if money > 0:
      shares = money / price_close
      df_money.loc[date_time] = 0
    else:
      shares = df_money.iloc[prev_index_num, 0] / price_close

    df_shares.loc[date_time] = shares

  elif rsi >= 70:
    if df_shares.iloc[prev_index_num, 0] == 0:
      df_money.loc[date_time] = df_money.iloc[prev_index_num, 0]
    else:
      return_money = df_shares.iloc[prev_index_num, 0] * price_close
      df_money.loc[date_time] = return_money

    df_shares.loc[date_time] = 0

  else:
    df_shares.loc[date_time] = df_shares.iloc[prev_index_num]
    df_money.loc[date_time] = df_money.iloc[prev_index_num]

display(df_shares)
display(df_money)

,shares
Date,
1997-01-06,15952.27
1997-02-04,15952.27
1997-03-06,14746.87
1997-04-04,14746.87
1997-05-07,14746.87
...,...
2023-05-31,0
2023-06-30,0
2023-07-31,0


,money
Date,
1997-01-06,0
1997-02-04,0
1997-03-06,0
1997-04-04,0
1997-05-07,0
...,...
2023-05-31,11184543.86
2023-06-30,11184543.86
2023-07-31,11184543.86
